In [1]:
# imports
import pandas as pd
from datetime import datetime, timedelta
import os
import numpy as np
from tqdm.notebook import tqdm

# constants
source_dir = "/content/drive/MyDrive/Leren en Beslissen/data lagersmit/raw data"
unsplit_dir = "/content/drive/MyDrive/Leren en Beslissen/data lagersmit/processed data/unsplit"
split_dir = "/content/drive/MyDrive/Leren en Beslissen/data lagersmit/processed data/split"
sampled_dir = "/content/drive/MyDrive/Leren en Beslissen/data lagersmit/processed data/sampled"
normalized_dir = "/content/drive/MyDrive/Leren en Beslissen/data lagersmit/processed data/normalized"
normalization_file = "/content/drive/MyDrive/Leren en Beslissen/data lagersmit/processed data/normalization.csv"

max_gap = timedelta(hours=1)
min_sequence = timedelta(days=1)
resample_time = timedelta(minutes = 5)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def load_files(subdir, files):
  raw_data = []

  for file in tqdm(files, desc="files"):
        if os.path.splitext(file)[1] == '.csv':
          print(f'- loading {file}')

          if (os.path.isfile(os.path.join(subdir, "_cleaned_" + file))):
            print("  cleaned alternative found. Skipping file")
          else:
            filepath = os.path.join(subdir, file)

            try:
              data = pd.read_csv(filepath, warn_bad_lines = True, error_bad_lines = False)

              data['Time'] = pd.to_datetime(data['Time'].astype(str), errors = 'coerce')
              data['File'] = file

              raw_data.append(data)

            except UnicodeDecodeError:
              print("  file is unreadable. Skipping file")
  if len(raw_data) > 0:
    return raw_data
  else:
    print("  no readable data data found. Skipping boat")
    return None

def clean_data(raw_data):
  data = raw_data.dropna()
  data = data[data['M20'] == 0]
  data.reset_index(inplace=True, drop=True)

  return data

def clean_and_merge_data(raw_data):
  print(f'- merging and cleaning {sum(map(lambda d : len(d.index), raw_data))} rows')
  
  # Clean unmerged file data
  data_list = list(filter(lambda data : not data.empty, map(clean_data, raw_data)))

  # Sort unmerged file data on start date
  data_list.sort(key = lambda d : d['Time'][0])

  # Merge file data into one DataFrame
  data = pd.concat(data_list, ignore_index = True)

  # Coerce all data (except Time and File) to numeric
  cols = data.columns.drop(['Time','File'])
  data[cols] = data[cols].apply(pd.to_numeric, errors='coerce')
  
  data.drop_duplicates(subset = 'Time', keep ='last', inplace = True)

  # Remove leftover unset dates
  data = data[data['M20'] == 0]

  # Reset index to ease selection into the DataFrame
  data.reset_index(inplace=True, drop=True)

  if len(data.index) > 100:
    print(f'  {len(data.index)} rows remaining')
    return data
  else:
    print(f'  {len(data.index)} rows remaining. Skipping boat')
    return None

def expand_data(data):
  print("- calculating expanded data")

  # get the time at the start of the data
  start_time = data['Time'][0]

  # convert the time to seconds since start of data en add a delta time in seconds
  data['TimeElapsed'] = (data['Time'] - start_time) / timedelta(seconds = 1)
  data['dt'] = data['TimeElapsed'].diff(1)

  if (len(data[data['dt'] < 0].index) > 0):
    print("  Boat is time traveling. Skipping boat")
    print(data[data['dt'] < 0][['Time', 'File', 'TimeElapsed', 'dt', 'M20']])
    
    return None

  else:
    data['Version'] = data.pop('B076.Cnt ').astype(int)

    drain = (data['Q2'] != data['Q2'].shift(1)) & (data['Q2'] == 1)
    data['autoEvent'] = (drain & (data['M22'] != 1))*1
    data['manualEvent'] = (drain & (data['M22'] == 1))*1

    data['autoCum'] = data['autoEvent'].cumsum()
    data['manualCum'] = data['manualEvent'].cumsum()
    data['totalCum'] = data['autoCum'] + data['manualCum']

    data['autoRolling'] = data.rolling('24h', on='Time')['autoEvent'].sum()
    data['manualRolling'] = data.rolling('24h', on='Time')['manualEvent'].sum()
    data['totalRolling'] = data['autoRolling'] + data['manualRolling']

    data['minLeakageCum'] = data['autoCum'] * 16
    data['maxLeakageCum'] = data['minLeakageCum'] + data['manualCum'] * 16
    data['averageLeakageCum'] = data['minLeakageCum'] + data['manualCum'] * 8

    data['minLeakageRolling'] = data['autoRolling'] * 16
    data['maxLeakageRolling'] = data['minLeakageRolling'] + data['manualRolling'] * 16
    data['averageLeakageRolling'] = data['minLeakageRolling'] + data['manualRolling'] * 8

    data['allowedLeakage'] = data['B099.Cnt '].astype(float) * 0.5

    return data

def split_data(data, name):
  print("- splitting data")
  split_data_sets = {}

  prev_index = 0
  number = 1
  
  split_indices = data[data['dt'] > max_gap.total_seconds()].index
  split_indices = split_indices.append(data.tail(1).index + 1)

  for cur_index in split_indices:
    split_data = data.loc[prev_index:cur_index - 1, :].reset_index(drop=True)
    duration = split_data.iloc[-1]['Time'] - split_data.iloc[0]['Time']

    if duration > min_sequence:
      days = duration // timedelta(days = 1)
      hours = duration % timedelta(days = 1) // timedelta(hours = 1)
      sub_name = f'{name} - {number} ({days} days {hours} hours)'

      split_data_sets[sub_name] = split_data

      print(f'  split chunk {number} with a duration of {duration}')

      number += 1

    prev_index = cur_index

  return split_data_sets

def save_data(name, dir, data):
  path = os.path.join(dir, name.replace("/", "_") + ".feather")
  print(f'  saving {name} to {path}')
  data.to_feather(path)

for subdir, dirs, files in tqdm(list(os.walk(source_dir)), desc="folder"):
    name = os.path.relpath(subdir, source_dir)
    print(f'\n{name}')

    raw_data = load_files(subdir, files)
    if raw_data is not None:
      cleaned_data = clean_and_merge_data(raw_data)

      if cleaned_data is not None:
        expanded_data = expand_data(cleaned_data)

        if expanded_data is not None:
          splitted_data_sets = split_data(expanded_data, name)

          print("- saving data")
          save_data(name, unsplit_dir, expanded_data)

          for subname, splitted_data in splitted_data_sets.items():
            save_data(subname, split_dir, splitted_data)
        

folder:   0%|          | 0/263 [00:00<?, ?it/s]


.


files:   0%|          | 0/7 [00:00<?, ?it/s]

- loading _3_4.csv
- merging and cleaning 20000 rows
  19996 rows remaining
- calculating expanded data
- splitting data
  split chunk 1 with a duration of 2 days 08:34:54
- saving data
  saving . to /content/drive/MyDrive/Leren en Beslissen/data lagersmit/processed data/unsplit/..feather
  saving . - 1 (2 days 8 hours) to /content/drive/MyDrive/Leren en Beslissen/data lagersmit/processed data/split/. - 1 (2 days 8 hours).feather

copies-70-b


files: 0it [00:00, ?it/s]

...

- loading _3_19.csv
- merging and cleaning 9 rows
  8 rows remaining. Skipping boat

copies-60-b/Roerborg - 9592599


files: 0it [00:00, ?it/s]

  no readable data data found. Skipping boat

copies-60-b/Roerborg - 9592599/X22847A


files:   0%|          | 0/4 [00:00<?, ?it/s]

- loading 20210824_3_40.csv
- loading 20210825_3_38.csv
- loading 20210825_3_37.csv
- loading 20210825_3_39.csv
- merging and cleaning 62649 rows
  62621 rows remaining
- calculating expanded data
- splitting data
  split chunk 1 with a duration of 7 days 03:02:34
  split chunk 2 with a duration of 3 days 03:46:33
- saving data
  saving copies-60-b/Roerborg - 9592599/X22847A to /content/drive/MyDrive/Leren en Beslissen/data lagersmit/processed data/unsplit/copies-60-b_Roerborg - 9592599_X22847A.feather
  saving copies-60-b/Roerborg - 9592599/X22847A - 1 (7 days 3 hours) to /content/drive/MyDrive/Leren en Beslissen/data lagersmit/processed data/split/copies-60-b_Roerborg - 9592599_X22847A - 1 (7 days 3 hours).feather
  saving copies-60-b/Roerborg - 9592599/X22847A - 2 (3 days 3 hours) to /content/drive/MyDrive/Leren en Beslissen/data lagersmit/processed data/split/copies-60-b_Roerborg - 9592599_X22847A - 2 (3 days 3 hours).feather


In [ ]:
def resample_data(data, timedelta):
  resampled_data = data.set_index("Time").resample(timedelta)
  
  resampled_data = resampled_data.ffill(limit=1)
  
  resampled_data.reset_index(inplace = True)
  
  resampled_data['dt'] = resampled_data['Time'].diff(1).apply(lambda dt : dt.total_seconds())

  resampled_data = resampled_data.dropna()

  resampled_data.reset_index(inplace = True, drop=True)
  
  return resampled_data

for subdir, dirs, files in tqdm(list(os.walk(split_dir)), desc="folders"):
  for file in tqdm(files, desc="files"):
    if os.path.splitext(file)[1] == '.feather':
      filepath = os.path.join(subdir, file)

      data = pd.read_feather(filepath)
      resampled_data = resample_data(data, resample_time)

      path = os.path.join(sampled_dir, file)
      resampled_data.to_feather(path)


folders:   0%|          | 0/1 [00:00<?, ?it/s]

files:   0%|          | 0/265 [00:00<?, ?it/s]

In [ ]:
totals = pd.Series(dtype='float64')
found = pd.Series(dtype='float64')

print("First pass")
for subdir, dirs, files in tqdm(list(os.walk(sampled_dir)), desc="folders"):
  for file in tqdm(files, desc="files"):
    if os.path.splitext(file)[1] == '.feather':
      filepath = os.path.join(subdir, file)

      data = pd.read_feather(filepath).select_dtypes(include=[np.number])
      index = totals.index.union(data.columns)

      totals = totals.reindex(index, fill_value=0) + data.sum().reindex(index, fill_value=0)
      found = found.reindex(index, fill_value=0) + data.count().reindex(index, fill_value=0)

index = found[found == found.max()].index
mean = totals/found
variance = pd.Series(dtype='float64').reindex(index, fill_value=0)

min = pd.Series(dtype='float64').reindex(index, fill_value=float('inf'))
max = pd.Series(dtype='float64').reindex(index, fill_value=float('-inf'))

print("Second pass")
for subdir, dirs, files in tqdm(list(os.walk(sampled_dir)), desc="folders"):
  for file in tqdm(files, desc="files"):
    if os.path.splitext(file)[1] == '.feather':
      filepath = os.path.join(subdir, file)
      data = pd.read_feather(filepath).reindex(index, axis=1)

      if all(elem in data for elem in index):
        variance += ((data - mean) ** 2).sum()

        max = data.append(max, ignore_index = True).max()
        min = data.append(min, ignore_index = True).min()

variance = variance / found.reindex(index)
std = variance ** 0.5

index = std[std != 0].index
index = index.drop(['B043.Cnt ', 'B098.AQ', 'B137.Ax', 'B211.Ax'])


mean = mean.reindex(index)
std = std.reindex(index)
max = max.reindex(index)
min = min.reindex(index)

print('Calculated info')
display(pd.DataFrame([mean, std, max, min], index=['mean', 'std', 'max', 'min']))

boolean_columns = (max == 1) & (min == 0)

shift = mean.copy()
shift[boolean_columns] = 0
divisor = std.copy()
divisor[boolean_columns] = 1

print('Normalization found')
normalize = pd.DataFrame([shift, divisor], index = ["shift", "divisor"])
normalize.to_csv(normalization_file)

display(normalize)


# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#   print()
#   print(std)

First pass


folders:   0%|          | 0/1 [00:00<?, ?it/s]

files:   0%|          | 0/265 [00:00<?, ?it/s]

Second pass


folders:   0%|          | 0/1 [00:00<?, ?it/s]

files:   0%|          | 0/265 [00:00<?, ?it/s]

Calculated info


,AI1,AI2,AI3,AI4,B005.AQ,B099.Cnt,B159.Cnt,B266.Cnt,I4,I5,I6,I7,M1,M10,M11,M12,M15,M2,M21,M22,M3,M4,M5,M6,M7,M8,M9,Q1,Q2,Q3,Q5,Q6,TimeElapsed,Version,allowedLeakage,autoCum,autoEvent,autoRolling,averageLeakageCum,averageLeakageRolling,manualCum,manualEvent,manualRolling,maxLeakageCum,maxLeakageRolling,minLeakageCum,minLeakageRolling,totalCum,totalRolling
mean,203.303031,310.340626,0.499472,0.480915,43.017703,503.958883,492.843257,1.505970,0.000133,0.906755,0.985273,0.000280,0.003116,0.000131,0.062962,0.932767,0.003625,0.006811,0.000217,0.000019,0.008723,0.003066,0.033691,0.031498,0.072631,0.000007,0.015698,0.077174,0.006337,0.883808,0.991277,0.966316,8.355767e+06,64.548854,251.979441,189.955575,0.001487,4.534339,3075.145768,73.119332,4.482071,0.000008,0.071239,3111.002337,73.689248,3039.289199,72.549417,194.437646,4.605578
std,78.163219,38.363059,0.814174,0.809035,86.141982,120.392767,1452.890883,0.832253,0.011516,0.290775,0.120460,0.016721,0.055730,0.011427,0.242895,0.250426,0.060095,0.082248,0.014728,0.004302,0.092986,0.055288,0.180433,0.174659,0.259529,0.002683,0.124305,0.266868,0.079353,0.320455,0.092986,0.180415,8.162394e+06,3.218777,60.196383,354.192097,0.038531,15.168541,5687.033949,242.990993,8.766249,0.002868,0.427868,5707.786259,243.333135,5667.073558,242.696649,356.736641,15.208321
max,672.000000,398.000000,5.000000,3.000000,778.000000,800.000000,13008.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5.336769e+07,73.000000,400.000000,1596.000000,1.000000,502.000000,25536.000000,8032.000000,60.000000,1.000000,9.000000,25536.000000,8032.000000,25536.000000,8032.000000,1596.000000,502.000000
min,0.000000,98.000000,0.000000,0.000000,-245.000000,240.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,60.000000,120.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Normalization found


,AI1,AI2,AI3,AI4,B005.AQ,B099.Cnt,B159.Cnt,B266.Cnt,I4,I5,I6,I7,M1,M10,M11,M12,M15,M2,M21,M22,M3,M4,M5,M6,M7,M8,M9,Q1,Q2,Q3,Q5,Q6,TimeElapsed,Version,allowedLeakage,autoCum,autoEvent,autoRolling,averageLeakageCum,averageLeakageRolling,manualCum,manualEvent,manualRolling,maxLeakageCum,maxLeakageRolling,minLeakageCum,minLeakageRolling,totalCum,totalRolling
shift,203.303031,310.340626,0.499472,0.480915,43.017703,503.958883,492.843257,1.505970,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.355767e+06,64.548854,251.979441,189.955575,0.0,4.534339,3075.145768,73.119332,4.482071,0.0,0.071239,3111.002337,73.689248,3039.289199,72.549417,194.437646,4.605578
divisor,78.163219,38.363059,0.814174,0.809035,86.141982,120.392767,1452.890883,0.832253,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,8.162394e+06,3.218777,60.196383,354.192097,1.0,15.168541,5687.033949,242.990993,8.766249,1.0,0.427868,5707.786259,243.333135,5667.073558,242.696649,356.736641,15.208321


In [ ]:
print(index)

Index(['AI1', 'AI2', 'AI3', 'AI4', 'B005.AQ  ', 'B099.Cnt ', 'B159.Cnt ',
       'B266.Cnt ', 'I4', 'I5', 'I6', 'I7', 'M1', 'M10', 'M11', 'M12', 'M15',
       'M2', 'M21', 'M22', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'Q1',
       'Q2', 'Q3', 'Q5', 'Q6', 'TimeElapsed', 'Version', 'allowedLeakage',
       'autoCum', 'autoEvent', 'autoRolling', 'averageLeakageCum',
       'averageLeakageRolling', 'manualCum', 'manualEvent', 'manualRolling',
       'maxLeakageCum', 'maxLeakageRolling', 'minLeakageCum',
       'minLeakageRolling', 'totalCum', 'totalRolling'],
      dtype='object')


In [ ]:
normalize = pd.read_csv(normalization_file, index_col=0)
display(normalize)

for subdir, dirs, files in tqdm(list(os.walk(sampled_dir)), desc="folders"):
  for file in tqdm(files, desc="files"):
    if os.path.splitext(file)[1] == '.feather':
      filepath = os.path.join(subdir, file)
      data = pd.read_feather(filepath).reindex(normalize.columns, axis=1)

      if len(data.index) > 1:
        normalized_data = (data - normalize.loc['shift'])/normalize.loc['divisor']
        
        path = os.path.join(normalized_dir, file)
        normalized_data.to_feather(path)



,AI1,AI2,AI3,AI4,B005.AQ,B099.Cnt,B159.Cnt,B266.Cnt,I4,I5,I6,I7,M1,M10,M11,M12,M15,M2,M21,M22,M3,M4,M5,M6,M7,M8,M9,Q1,Q2,Q3,Q5,Q6,TimeElapsed,Version,allowedLeakage,autoCum,autoEvent,autoRolling,averageLeakageCum,averageLeakageRolling,manualCum,manualEvent,manualRolling,maxLeakageCum,maxLeakageRolling,minLeakageCum,minLeakageRolling,totalCum,totalRolling
shift,203.303031,310.340626,0.499472,0.480915,43.017703,503.958883,492.843257,1.505970,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.355767e+06,64.548854,251.979441,189.955575,0.0,4.534339,3075.145768,73.119332,4.482071,0.0,0.071239,3111.002337,73.689248,3039.289199,72.549417,194.437646,4.605578
divisor,78.163219,38.363059,0.814174,0.809035,86.141982,120.392767,1452.890883,0.832253,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,8.162394e+06,3.218777,60.196383,354.192097,1.0,15.168541,5687.033949,242.990993,8.766249,1.0,0.427868,5707.786259,243.333135,5667.073558,242.696649,356.736641,15.208321


folders:   0%|          | 0/1 [00:00<?, ?it/s]

files:   0%|          | 0/265 [00:00<?, ?it/s]

In [ ]:
normalize.shape

(2, 49)